## Approach A

In [1]:
import os
import torch
import torchaudio
import pandas as pd

def load_and_preprocess_audios_from_folder(folder_path, target_sr=16000):
    """
    Load and normalize all audio files in a folder using torchaudio, extracting accent and gender from filename.

    Args:
        folder_path (str): Path to folder containing .wav files
        target_sr (int): Sampling rate

    Returns:
        pd.DataFrame: DataFrame with columns ['file_path', 'waveform', 'accent', 'gender']
    """
    data = []
    for fname in os.listdir(folder_path):
        if fname.endswith('.wav'):
            file_path = os.path.join(folder_path, fname)
            # Load audio
            waveform, sr = torchaudio.load(file_path)
            # Normalize amplitude
            waveform = waveform / waveform.abs().max()
            # Extract accent and gender
            accent = int(fname[0])  # 1-5
            gender = fname[1]       # 'm' or 'f'
            data.append({
                'file_path': file_path,
                'waveform': waveform,
                'accent': accent,
                'gender': gender
            })
    return pd.DataFrame(data)

df = load_and_preprocess_audios_from_folder("/Users/larsheijnen/DL/Train")
print(df.head())

#Size first waveform
print(df['waveform'].iloc[0].shape)


                                 file_path  \
0  /Users/larsheijnen/DL/Train/2m_9039.wav   
1  /Users/larsheijnen/DL/Train/4f_1887.wav   
2  /Users/larsheijnen/DL/Train/4f_9571.wav   
3  /Users/larsheijnen/DL/Train/1m_3736.wav   
4  /Users/larsheijnen/DL/Train/1m_3078.wav   

                                            waveform  accent gender  
0  [[tensor(-0.0001), tensor(-0.0001), tensor(-5....       2      m  
1  [[tensor(0.), tensor(4.4749e-05), tensor(0.), ...       4      f  
2  [[tensor(-0.0001), tensor(-0.0002), tensor(-0....       4      f  
3  [[tensor(-0.0003), tensor(-0.0003), tensor(-0....       1      m  
4  [[tensor(-0.0008), tensor(-0.0009), tensor(-0....       1      m  
torch.Size([1, 41400])


In [2]:
import os
import torch
import torchaudio
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Convert accent from 1-5 to 0-4
if 'accent' in df.columns:
    df['accent_label'] = df['accent'].apply(lambda x: int(x) - 1)
else:
    print("Warning: 'accent' column not found in DataFrame. Accent prediction will not work.")
    # Add a dummy label if you want to proceed with model structure testing
    df['accent_label'] = 0

In [3]:
# --- 1. Device Configuration ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Voor macos met M-chips

Using device: mps


In [4]:
# --- 2. PyTorch Dataset ---
class AccentDatasetRNN(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Waveform is already a tensor [1, length] from your preprocessing
        waveform = self.df.iloc[idx]['waveform'].squeeze(0) # RNN expects [seq_len, features] or [batch, seq_len, features]
                                                           # Here, seq_len is audio length, features is 1 (mono)
        accent_label = self.df.iloc[idx]['accent']
        # Convert accent 1-5 to 0-4 for CrossEntropyLoss
        label = torch.tensor(accent_label - 1, dtype=torch.long)
        return waveform, label

In [5]:
# --- 3. Custom Collate Function ---
def collate_fn_rnn(batch):
    # batch is a list of tuples (waveform_tensor, label_tensor)
    waveforms = [item[0] for item in batch]
    labels = torch.stack([item[1] for item in batch])
    
    # Get lengths of each sequence
    lengths = torch.tensor([len(w) for w in waveforms])

    # Pad sequences in this batch (batch_first=False for pack_padded_sequence easier handling with RNN)
    # pad_sequence expects a list of tensors, each tensor is a sequence
    padded_waveforms = pad_sequence(waveforms, batch_first=False, padding_value=0.0)
    # padded_waveforms will be of shape (max_seq_len_in_batch, batch_size)
    # We need it as (max_seq_len_in_batch, batch_size, num_features=1) for RNN
    padded_waveforms = padded_waveforms.unsqueeze(-1) # Add feature dimension

    return padded_waveforms, lengths, labels

In [6]:
# --- 4. RNN Model Definition ---
class AudioRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, num_classes=5, rnn_type='LSTM', dropout=0.3):
        super(AudioRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn_type = rnn_type.upper()

        if self.rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, 
                               batch_first=False, dropout=dropout if num_layers > 1 else 0)
        elif self.rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers,
                              batch_first=False, dropout=dropout if num_layers > 1 else 0)
        else:
            raise ValueError("Unsupported RNN type. Choose 'LSTM' or 'GRU'.")
        
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths):
        # x shape: (max_seq_len, batch_size, input_size)
        # lengths shape: (batch_size)

        # Pack sequence
        # Enforce_sorted=False because DataLoader with shuffle=True might not guarantee order
        # Sorting is done within collate_fn if needed, but pack_padded_sequence can handle unsorted if lengths are provided
        packed_input = pack_padded_sequence(x, lengths.cpu(), batch_first=False, enforce_sorted=False)
        
        # RNN forward pass
        if self.rnn_type == 'LSTM':
            packed_output, (hn, cn) = self.rnn(packed_input)
        else: # GRU
            packed_output, hn = self.rnn(packed_input)
        
        # We can use the hidden state of the last layer (hn)
        # hn is (num_layers, batch_size, hidden_size)
        # We want the output of the last RNN layer for each sequence
        # Take the hidden state of the last layer:
        last_hidden = hn[-1] # Shape: (batch_size, hidden_size)
        
        out = self.dropout(last_hidden)
        out = self.fc(out)
        return out

In [ ]:
# --- 5. Training Setup ---
# Hyperparameters
input_size = 1  # Mono audio
hidden_size = 128 # Can be tuned
num_rnn_layers = 2 # Can be tuned
num_classes = 5 # 5 accent classes
learning_rate = 0.001
batch_size = 16 # Adjust based on your 8GB RAM. Start small.
num_epochs = 10 # Start with a few epochs

# Instantiate dataset and dataloader
# Ensure 'df' is your DataFrame with 'waveform' and 'accent' columns
if 'df' not in locals():
    print("DataFrame 'df' not found. Please load your data.")
    # df = load_and_preprocess_audios_from_folder("/Users/larsheijnen/DL/Train") # Placeholder
else:
    train_dataset = AccentDatasetRNN(df)
    train_loader = DataLoader(dataset=train_dataset, 
                              batch_size=batch_size, 
                              shuffle=True, 
                              collate_fn=collate_fn_rnn,
                              pin_memory=False, # pin_memory is more relevant for CUDA
                              num_workers=0 # <--- CHANGE THIS
                             )

    # Instantiate model, loss, and optimizer
    model = AudioRNN(input_size=input_size, 
                     hidden_size=hidden_size, 
                     num_layers=num_rnn_layers, 
                     num_classes=num_classes,
                     rnn_type='GRU' # GRU is often a bit faster and lighter than LSTM
                    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"Starting training on {device} with batch size {batch_size}...")

    # --- Training Loop ---
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for i, (waveforms, lengths, labels) in enumerate(train_loader):
            waveforms = waveforms.to(device) 
            # lengths are already tensors, should be on CPU for pack_padded_sequence
            labels = labels.to(device)

            # Forward pass
            outputs = model(waveforms, lengths)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if (i + 1) % 10 == 0: # Print every 10 batches
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}] completed. Average Training Loss: {avg_loss:.4f}')

    print('Finished Training')

Starting training on mps with batch size 16...
